## Get the playlists and tracks from the users Ryan gave in his text file

Note: takes about 8-9 hours to run through 8000 users.

##### Column info

collaborative returns t/f
description returns text
take nothing from external_urls
followers['total'] is the int
nothing from href
id is the id
nothing from image
text from name
can get username from my query not from owner
['tracks']['total'] gives you number of tracks

tracks data frame
from tracks items ['added_at'] get the data time the track was added
from tracks items ['track'] get duration_ms as float
for artist in tracks items ['track']['artists'][0]['name'] get the first artist
from tracks items ['track'] get explicit as bool
from tracks items ['track'] get id as string
from tracks items ['track'] get name as string
from tracks items ['track'] get popularity as float (0-100)

In [1]:
# imports
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context('paper')
sns.color_palette('muted')
%matplotlib inline

In [45]:
# Get the data
plists = pd.read_json('data/plist_df.json')
tracks = pd.read_json('data/track_df.json')


In [2]:
user_followers = pd.read_json('getplaylist/user_follow.json')

8457